In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models



In [5]:
# 数据预处理和加载
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # 调整大小以适应预训练模型
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

# 加载预训练的 ResNet18 模型
model = models.resnet18(pretrained=True)

# 修改模型以适应 MNIST 数据集（单通道输入和10类输出）
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
#优化以下配置，重新运行，可将准确率提升到>99%
# model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
# print(model.fc.in_features)  512
model.fc = nn.Linear(model.fc.in_features, 10)

# 使用 GPU (如果可用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib64/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib64/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


512


In [3]:

# 训练模型
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

print("training finished")


Epoch [1/5], Step [0/938], Loss: 2.6610
Epoch [1/5], Step [100/938], Loss: 0.1249
Epoch [1/5], Step [200/938], Loss: 0.1127
Epoch [1/5], Step [300/938], Loss: 0.0784
Epoch [1/5], Step [400/938], Loss: 0.2221
Epoch [1/5], Step [500/938], Loss: 0.1883
Epoch [1/5], Step [600/938], Loss: 0.0313
Epoch [1/5], Step [700/938], Loss: 0.0274
Epoch [1/5], Step [800/938], Loss: 0.0363
Epoch [1/5], Step [900/938], Loss: 0.1166
Epoch [2/5], Step [0/938], Loss: 0.0336
Epoch [2/5], Step [100/938], Loss: 0.0832
Epoch [2/5], Step [200/938], Loss: 0.0506
Epoch [2/5], Step [300/938], Loss: 0.0260
Epoch [2/5], Step [400/938], Loss: 0.0401
Epoch [2/5], Step [500/938], Loss: 0.0596
Epoch [2/5], Step [600/938], Loss: 0.1334
Epoch [2/5], Step [700/938], Loss: 0.1417
Epoch [2/5], Step [800/938], Loss: 0.0851
Epoch [2/5], Step [900/938], Loss: 0.0648
Epoch [3/5], Step [0/938], Loss: 0.0295
Epoch [3/5], Step [100/938], Loss: 0.1579
Epoch [3/5], Step [200/938], Loss: 0.0478
Epoch [3/5], Step [300/938], Loss: 0.081

In [4]:
# 测试模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total} %')


Accuracy of the model on the test images: 98.93 %
